### Pyspark kernel
allows you to now have to run %%spark in every cell.
Once a livy session is started, all cells communicate to that livy


In [2]:
%load_ext sparkmagic.magics

#### Py virtual env

In [3]:
%%spark config 
{
  "name":"Parquet_dump",
  "driverMemory":"2G",
  "numExecutors":1,
  "proxyUser":"noobie",
  "archives": ["/user/noobie/gethdemo.tar.gz"],
  "files": ["/user/noobie/etherdelta_abi.json"],
  "queue": "noobies",
  "conf": {"spark.yarn.appMasterEnv.PYSPARK_PYTHON":"gethdemo.tar.gz/demo/bin/python3.5",
          "spark.jars.packages":"org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.0"}
}

In [4]:
%spark add -s ethlogsdw -l python -u http://hdp-3.evilcorp.trade:8999 --auth Kerberos

Skip
Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
313,application_1529358819933_0003,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


## Start consumer


In [5]:
%%spark
from kafka import KafkaConsumer, KafkaProducer

consumer = KafkaConsumer(bootstrap_servers=['hdp-3.evilcorp.trade:6667',
                                           'hdp-5.evilcorp.trade:6667',
                                           'hdp-6.evilcorp.trade:6667'],
                        security_protocol="SASL_PLAINTEXT",
                        sasl_mechanism="GSSAPI",
                        auto_offset_reset='earliest',
                        client_id='SparlSqlEthLogs',
                        request_timeout_ms=501,
                        consumer_timeout_ms=500,
                        max_poll_interval_ms=500)
#,session_timeout_ms=500
ETH_KAFKA_TOPIC='selfpump_trades_sat'
ETH_KAFKA_TOPIC='selfpump_trades_stream'

In [6]:
%%spark
consumer.subscribe([ETH_KAFKA_TOPIC])

Sample data in consumermsg
```
{'transactionIndex': 68, 
           'logIndex': 63, 
           'blockHash': '0x9aa8982af8df5b99e967bc59694a64bb27bdc6f689cad01096b7e894e5e01a0a', 
           'event': 'Trade', 
           'args': {'get': '0x01DD9cD663A17f72115304F75014D169Ae5E6494', 
                    'tokenGet': '0x9214eC02CB71CbA0ADA6896b8dA260736a67ab10', 
                    'tokenGive': '0x0000000000000000000000000000000000000000', 
                    'amountGive': 8512000000000000, 'amountGet': 16000000000000000000, 
                    'give': '0x01DD9cD663A17f72115304F75014D169Ae5E6494'}, 
           'transactionHash': '0x651c4b6ad804415bb4155ea115d9927a011d9b2edc3d3a2eb21738ccfa3eeb50', 
           'address': '0x8d12A197cB00D4747a1fe03395095ce2A5CC6819', 
           'blockNumber': 5684183}
```

In [ ]:
%%spark
import json
from pyspark.sql import HiveContext
from pyspark.sql.types import DecimalType

hc = HiveContext(sc)

ed_trades_dict = [ ]
for message in consumer:
    consumermsg=json.loads(message.value.decode('utf-8'))   
    ed_trades_dict.append(consumermsg)
    
nested_df = hc.read.json(sc.parallelize(ed_trades_dict))
pumps_df = nested_df.select(nested_df.blockNumber, 
                           nested_df.event,
                           nested_df.args.amountGet.cast(DecimalType(38,2)).alias("amountGet"),
                           nested_df.args.amountGive.cast(DecimalType(38,2)).alias("amountGive"),
                           nested_df.args.get.alias("buyer"),
                           nested_df.args.give.alias("seller"),
                           nested_df.args.tokenGive.alias("tokenGive"),
                           nested_df.args.tokenGet.alias("tokenGet"),
                           nested_df.transactionHash)
                           #"blockNumber", "event", "args.*", "address", "transactionHash")
#pumps_df.collect()


In [ ]:
%%spark
pumps_df.printSchema()

In [163]:
%%spark
pumps_df.createOrReplaceTempView("SelfPumpToday")
#pumps_df.write.csv("/user/noobie/visualization_data.csv")

#pumps_df.write.parquet("/user/noobie/pumps.parquet")

In [ ]:
%%spark -c sql
select count(*) from SelfPumpToday

In [132]:
# fix sparkmagic output https://github.com/jupyter-incubator/sparkmagic/issues/319
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [133]:
%%spark -c sql 
select seller, buyer, amountGive, transactionHash from SelfPumpToday where tokenGive LIKE '0x000000000000000%' \
order by amountGive desc

                                        buyer  \
0  0xAFB7340211f56618789858A73D34a67e837624C4   
1  0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A   
2  0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A   

                                       seller  \
0  0xAFB7340211f56618789858A73D34a67e837624C4   
1  0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A   
2  0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A   

                                                      transactionHash  \
0  0x065499b3d3467e82b745b0403c4c5eba9e8222d1acbbb967ab34dbf00fe42bef   
1  0x4d0548d15e7f4518810e258545d3135f02662c4bb23b0da9d32e5824c95313ce   
2  0x171ff7ec5f05e4d7c0d8a8f7b94fa13b65a6a330ffe6b7689d52fd1f4c25c1d4   

     amountGive  
0  7.350000e+16  
1  4.727799e+16  
2  3.011100e+16  


,buyer,seller,transactionHash,amountGive
0,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x065499b3d3467e82b745b0403c4c5eba9e8222d1acbbb967ab34dbf00fe42bef,7.350000e+16
1,0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A,0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A,0x4d0548d15e7f4518810e258545d3135f02662c4bb23b0da9d32e5824c95313ce,4.727799e+16
2,0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A,0x1a0A311527414A94c971104DBC8dd84b0ecc1d4A,0x171ff7ec5f05e4d7c0d8a8f7b94fa13b65a6a330ffe6b7689d52fd1f4c25c1d4,3.011100e+16


In [136]:
%%spark -c sql -o marketmanipulated
select buyer, seller, amountGet, transactionHash from SelfPumpToday where tokenGet LIKE '0x000000000000000%' \
order by amountGet desc

                                         buyer  \
0   0xAFB7340211f56618789858A73D34a67e837624C4   
1   0x09e2D651cE800B048611B5DFca8613452F670F7d   
2   0x09e2D651cE800B048611B5DFca8613452F670F7d   
3   0x09e2D651cE800B048611B5DFca8613452F670F7d   
4   0x09e2D651cE800B048611B5DFca8613452F670F7d   
5   0x09e2D651cE800B048611B5DFca8613452F670F7d   
6   0xAFB7340211f56618789858A73D34a67e837624C4   
7   0xAFB7340211f56618789858A73D34a67e837624C4   
8   0xAFB7340211f56618789858A73D34a67e837624C4   
9   0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0   
10  0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0   
11  0xAFB7340211f56618789858A73D34a67e837624C4   
12  0xf1C7cA754F5Cd2DF308d19632763d292cF406060   
13  0xf1C7cA754F5Cd2DF308d19632763d292cF406060   
14  0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0   
15  0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0   
16  0xAFB7340211f56618789858A73D34a67e837624C4   
17  0xAFB7340211f56618789858A73D34a67e837624C4   
18  0xAFB7340211f56618789858A73D34a67e837624C4   


,buyer,seller,transactionHash,amountGet
0,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0xfdbf2ac5aea9ce211d5ea9a4cb3bf614f9fafa9f798fd17d53246cc23c223ee6,5.270000e+18
1,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xe103223d7677973e2ef563b9ce02eddd19edbecbcdf653f6b6945556c4a3685a,3.050000e+18
2,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xe103223d7677973e2ef563b9ce02eddd19edbecbcdf653f6b6945556c4a3685a,3.050000e+18
3,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb70817e327930e8be0a97a2b78024c7e1aaa38112622a7442bf8c0aa3d7614c2,1.860000e+18
4,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb70817e327930e8be0a97a2b78024c7e1aaa38112622a7442bf8c0aa3d7614c2,1.860000e+18
5,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb4f8b7f17a107b222bccf53c2765b87dfdf050373b9da77c842ffde17ed17291,1.049000e+18
6,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x860fa1785201135caf8e5740f7463e4233cecb2297bd4565fc0f7030b0a63eb2,8.100000e+17
7,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x860fa1785201135caf8e5740f7463e4233cecb2297bd4565fc0f7030b0a63eb2,8.100000e+17
8,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x8139ceb1e4955e3674bdda2639eab0a250d02a8f9015981cd24db3f0426af98e,7.200000e+17
9,0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0,0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0,0x04c7418ec98ad67beb7cba9c5964c4bedef745d87c752d21965fa0a46b7492b7,5.549000e+17


In [137]:
marketmanipulated

,buyer,seller,transactionHash,amountGet
0,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0xfdbf2ac5aea9ce211d5ea9a4cb3bf614f9fafa9f798fd17d53246cc23c223ee6,5.270000e+18
1,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xe103223d7677973e2ef563b9ce02eddd19edbecbcdf653f6b6945556c4a3685a,3.050000e+18
2,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xe103223d7677973e2ef563b9ce02eddd19edbecbcdf653f6b6945556c4a3685a,3.050000e+18
3,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb70817e327930e8be0a97a2b78024c7e1aaa38112622a7442bf8c0aa3d7614c2,1.860000e+18
4,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb70817e327930e8be0a97a2b78024c7e1aaa38112622a7442bf8c0aa3d7614c2,1.860000e+18
5,0x09e2D651cE800B048611B5DFca8613452F670F7d,0x09e2D651cE800B048611B5DFca8613452F670F7d,0xb4f8b7f17a107b222bccf53c2765b87dfdf050373b9da77c842ffde17ed17291,1.049000e+18
6,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x860fa1785201135caf8e5740f7463e4233cecb2297bd4565fc0f7030b0a63eb2,8.100000e+17
7,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x860fa1785201135caf8e5740f7463e4233cecb2297bd4565fc0f7030b0a63eb2,8.100000e+17
8,0xAFB7340211f56618789858A73D34a67e837624C4,0xAFB7340211f56618789858A73D34a67e837624C4,0x8139ceb1e4955e3674bdda2639eab0a250d02a8f9015981cd24db3f0426af98e,7.200000e+17
9,0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0,0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0,0x04c7418ec98ad67beb7cba9c5964c4bedef745d87c752d21965fa0a46b7492b7,5.549000e+17


In [150]:
%%spark -c sql -o pew
select seller, sum(amountGet) as total, count(transactionHash) as num_trans \
from SelfPumpToday where tokenGet LIKE '0x000000000000000%' \
group by seller \
order by total, num_trans desc

    num_trans                                      seller         total
0           1  0xbc1323B62Bdd371C38A5f7904c705d833A836d2E  8.800000e+13
1           2  0x8Dd44F5ef0d1765a428616E943351d009D7B1088  8.251409e+14
2           1  0x2ce8F2EA67288e90339bCB4a1B883a9f78bCA78c  1.000000e+15
3           1  0x6DBD7240D360BE22CBa37e207a727e5a59FbaAAF  1.400000e+15
4           1  0xF23db283c6b43feCe3b1215cF5507B6d78fc2215  2.990729e+15
5           2  0x6ec8247cdc03F336118B96f4e862b24B1F947fa2  7.890000e+15
6           1  0xa81EB39A59fB926c81F260A6539e1C7e85fa71b9  7.048800e+16
7           2  0xe84a294E045535e39CFFc81D2Dc74b2839035388  8.306368e+16
8           2  0x736a1120d12f1D19E0cEF35426C7ec7B90514be4  1.137972e+17
9           1  0x73ef6A3b70CD95C775Faa1eEF0691AA6800584e6  1.279875e+17
10          1  0x4D25819e9De2A2EEEdDca953D1eF0182680a7054  1.420000e+17
11          3  0xf1C7cA754F5Cd2DF308d19632763d292cF406060  8.096400e+17
12          4  0x1c17d6CEF711ea2b1B3dF19ffa5E8Efc1467daE0  1.575

,num_trans,seller,total
0,1,0xbc1323B62Bdd371C38A5f7904c705d833A836d2E,8.800000e+13
1,2,0x8Dd44F5ef0d1765a428616E943351d009D7B1088,8.251409e+14
2,1,0x2ce8F2EA67288e90339bCB4a1B883a9f78bCA78c,1.000000e+15
3,1,0x6DBD7240D360BE22CBa37e207a727e5a59FbaAAF,1.400000e+15
4,1,0xF23db283c6b43feCe3b1215cF5507B6d78fc2215,2.990729e+15
5,2,0x6ec8247cdc03F336118B96f4e862b24B1F947fa2,7.890000e+15
6,1,0xa81EB39A59fB926c81F260A6539e1C7e85fa71b9,7.048800e+16
7,2,0xe84a294E045535e39CFFc81D2Dc74b2839035388,8.306368e+16
8,2,0x736a1120d12f1D19E0cEF35426C7ec7B90514be4,1.137972e+17
9,1,0x73ef6A3b70CD95C775Faa1eEF0691AA6800584e6,1.279875e+17


In [151]:
pew

,num_trans,seller,total
0,1,0xbc1323B62Bdd371C38A5f7904c705d833A836d2E,8.800000e+13
1,2,0x8Dd44F5ef0d1765a428616E943351d009D7B1088,8.251409e+14
2,1,0x2ce8F2EA67288e90339bCB4a1B883a9f78bCA78c,1.000000e+15
3,1,0x6DBD7240D360BE22CBa37e207a727e5a59FbaAAF,1.400000e+15
4,1,0xF23db283c6b43feCe3b1215cF5507B6d78fc2215,2.990729e+15
5,2,0x6ec8247cdc03F336118B96f4e862b24B1F947fa2,7.890000e+15
6,1,0xa81EB39A59fB926c81F260A6539e1C7e85fa71b9,7.048800e+16
7,2,0xe84a294E045535e39CFFc81D2Dc74b2839035388,8.306368e+16
8,2,0x736a1120d12f1D19E0cEF35426C7ec7B90514be4,1.137972e+17
9,1,0x73ef6A3b70CD95C775Faa1eEF0691AA6800584e6,1.279875e+17


## Store to parquet

In [ ]:
%%spark
pumps_df.write.parquet("/tmp/test.parquet")
